In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from net import FraudNet, AttentionTransformerFraudNet, EnhancedFraudNet  # Import fraud detection model
from data import get_dataloaders_fraud  # Import dataset functions
from evaluation import evaluate_model  # Import evaluation function
from train import train_model, set_all_seeds  # Import training function from train.py
import pandas as pd
import sys
from plot import plot_metrics, plot_confusion_matrices, plot_aucpr
import pickle


# Load fraud dataset
set_all_seeds(42)

# Set dataset path
DATASET_PATH = "/home/khoa/Khoa/outsource/na_thesis/examples/hello-world/ml-to-fl/pt/src/data/creditcard.csv"
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

# Training hyperparameters
batch_size = 32
num_epochs = 15
learning_rate = 0.00003

df = pd.read_csv(DATASET_PATH)
input_size = df.shape[1] - 1
print(f"Detected input size: {input_size}")

train_loader, valid_loader, test_loader, class_weights = get_dataloaders_fraud(
    DATASET_PATH, batch_size=batch_size, use_smote=True, plot=True, save_plot_dir='data_plot'
)

In [ ]:
for model in [AttentionTransformerFraudNet(input_size=input_size).to(DEVICE), FraudNet(input_size=input_size).to(DEVICE), EnhancedFraudNet(input_size=input_size).to(DEVICE)]:
    # Get model name for saving metrics
    model_name = model.__class__.__name__

    class_weights = class_weights
    pos_weight = torch.tensor([class_weights[1] / class_weights[0]], device=DEVICE)

    pos_weight = None

    # Loss Function (No weight balancing since using SMOTE)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Call `train.py` instead of writing the training loop here
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', patience=3, verbose=True
    )

    train_loss_list, train_metrics_list, valid_metrics_list, test_metrics = train_model(
        model, num_epochs, train_loader, valid_loader, test_loader, optimizer,
        criterion, DEVICE, scheduler=scheduler, stochastic=False
    )

    if pos_weight:
        save_plot_dir = f'plot_{model_name}_{batch_size}_{num_epochs}_{learning_rate}_pos_weight'
    else:
        save_plot_dir = f'plot_{model_name}_{batch_size}_{num_epochs}_{learning_rate}'
        
    os.makedirs(save_plot_dir, exist_ok=True)

    # Create metrics directory if it doesn't exist
    metrics_dir = f'metrics'
    os.makedirs(metrics_dir, exist_ok=True)

    # Save training metrics
    metrics_data = {
        'train_metrics': train_metrics_list,
        'valid_metrics': valid_metrics_list,
        'test_metrics': test_metrics,
        'train_loss': train_loss_list
    }
    if pos_weight:
        metrics_file = os.path.join(metrics_dir, f"{model_name}_{batch_size}_{num_epochs}_{learning_rate}_pos_weight_metrics.pickle")
    else:
        metrics_file = os.path.join(metrics_dir, f"{model_name}_{batch_size}_{num_epochs}_{learning_rate}_metrics.pickle")
        
    with open(metrics_file, 'wb') as f:
        pickle.dump(metrics_data, f)

    print(f"Metrics saved to {metrics_file}")

    # Create plots
    plot_metrics(train_metrics_list, fig_name="Training Metrics", save_path=f"{save_plot_dir}/train_metrics.png")
    plot_metrics(valid_metrics_list, fig_name="Validation Metrics", save_path=f"{save_plot_dir}/valid_metrics.png")
    plot_confusion_matrices(model, test_loader, threshold=0.85, save_path=f"{save_plot_dir}/confusion_matrix.png")
    plot_aucpr(model, test_loader, device=DEVICE, save_path=f"{save_plot_dir}/auc_pr.png")

    # Save the trained model
    best_model_path = "best_model.pth"
    model.load_state_dict(torch.load(best_model_path))
    print("Loaded best model from training phase.")

    # Save the best model explicitly at a clear location for future usage
    if pos_weight:
        final_model_path = f"./best_{model_name}_{batch_size}_{num_epochs}_{learning_rate}_pos_weight_model.pth"
    else:
        final_model_path = f"./best_{model_name}_{batch_size}_{num_epochs}_{learning_rate}_model.pth"
        
    torch.save(model.state_dict(), final_model_path)
    print(f"Final best model saved explicitly at {final_model_path}")

    # Evaluate model
    print("Evaluating Model on Test Set...")
    evaluate_model(model, test_loader, DEVICE)

# AIO

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from net import FraudNet, AttentionTransformerFraudNet, EnhancedFraudNet  # Import fraud detection model
from data import get_dataloaders_fraud  # Import dataset functions
from evaluation import evaluate_model  # Import evaluation function
from train import train_model, set_all_seeds  # Import training function from train.py
import pandas as pd
import sys
from plot import plot_metrics, plot_confusion_matrices, plot_aucpr
import pickle


# Load fraud dataset
set_all_seeds(42)

# Set dataset path
DATASET_PATH = "/home/khoa/Khoa/outsource/na_thesis/examples/hello-world/ml-to-fl/pt/src/data/creditcard.csv"
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

# Training hyperparameters
batch_size = 32

for batch_size in [96, 128, 256]:
    num_epochs = 15
    learning_rate = 0.00003

    df = pd.read_csv(DATASET_PATH)
    input_size = df.shape[1] - 1
    print(f"Detected input size: {input_size}")

    train_loader, valid_loader, test_loader, class_weights = get_dataloaders_fraud(
        DATASET_PATH, batch_size=batch_size, use_smote=True, plot=True, save_plot_dir='data_plot'
    )

    for model in [AttentionTransformerFraudNet(input_size=input_size).to(DEVICE), FraudNet(input_size=input_size).to(DEVICE), EnhancedFraudNet(input_size=input_size).to(DEVICE)]:
        for pos_weight in [torch.tensor([class_weights[1] / class_weights[0]], device=DEVICE), None]:
            # Get model name for saving metrics
            model_name = model.__class__.__name__

            # Loss Function (No weight balancing since using SMOTE)
            criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

            # Optimizer
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)

            # Call `train.py` instead of writing the training loop here
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode='max', patience=3, verbose=True
            )

            train_loss_list, train_metrics_list, valid_metrics_list, test_metrics = train_model(
                model, num_epochs, train_loader, valid_loader, test_loader, optimizer,
                criterion, DEVICE, scheduler=scheduler, stochastic=False
            )

            if pos_weight:
                save_plot_dir = f'plot_{model_name}_{batch_size}_{num_epochs}_{learning_rate}_pos_weight'
            else:
                save_plot_dir = f'plot_{model_name}_{batch_size}_{num_epochs}_{learning_rate}'
                
            os.makedirs(save_plot_dir, exist_ok=True)

            # Create metrics directory if it doesn't exist
            metrics_dir = f'metrics'
            os.makedirs(metrics_dir, exist_ok=True)

            # Save training metrics
            metrics_data = {
                'train_metrics': train_metrics_list,
                'valid_metrics': valid_metrics_list,
                'test_metrics': test_metrics,
                'train_loss': train_loss_list
            }
            if pos_weight:
                metrics_file = os.path.join(metrics_dir, f"{model_name}_{batch_size}_{num_epochs}_{learning_rate}_pos_weight_metrics.pickle")
            else:
                metrics_file = os.path.join(metrics_dir, f"{model_name}_{batch_size}_{num_epochs}_{learning_rate}_metrics.pickle")
                
            with open(metrics_file, 'wb') as f:
                pickle.dump(metrics_data, f)

            print(f"Metrics saved to {metrics_file}")

            # Create plots
            plot_metrics(train_metrics_list, fig_name="Training Metrics", save_path=f"{save_plot_dir}/train_metrics.png")
            plot_metrics(valid_metrics_list, fig_name="Validation Metrics", save_path=f"{save_plot_dir}/valid_metrics.png")
            plot_confusion_matrices(model, test_loader, threshold=0.85, save_path=f"{save_plot_dir}/confusion_matrix.png")
            plot_aucpr(model, test_loader, device=DEVICE, save_path=f"{save_plot_dir}/auc_pr.png")

            # Save the trained model
            best_model_path = "best_model.pth"
            model.load_state_dict(torch.load(best_model_path))
            print("Loaded best model from training phase.")

            # Save the best model explicitly at a clear location for future usage
            if pos_weight:
                final_model_path = f"./best_{model_name}_{batch_size}_{num_epochs}_{learning_rate}_pos_weight_model.pth"
            else:
                final_model_path = f"./best_{model_name}_{batch_size}_{num_epochs}_{learning_rate}_model.pth"
                
            torch.save(model.state_dict(), final_model_path)
            print(f"Final best model saved explicitly at {final_model_path}")

            # Evaluate model
            print("Evaluating Model on Test Set...")
            evaluate_model(model, test_loader, DEVICE)

In [ ]:
import pickle

DIR = '/home/khoa/Khoa/outsource/na_thesis/examples/hello-world/ml-to-fl/pt/src/metrics'
metrics = os.listdir(DIR)
metrics.sort()

for metric in metrics:
    if metric.startswith('Attention') and 'pos_weight' not in metric:
        with open(os.path.join(DIR, metric), 'rb') as f:
            data = pickle.load(f)
        
        print(data.keys())
        print(data['train_metrics'][1].keys())


# Note
## train_loss for comparison between model (same batch size)

# Compare Model with itself in diff batch

In [ ]:
import os
import pickle
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import savgol_filter
import re

def sort_by_model_and_size(filename):
    # Extract model name (everything before the first underscore)
    model_match = re.match(r'([^_]+)_', filename)
    model_name = model_match.group(1) if model_match else ""
    
    # Extract size (first number after the model name)
    size_match = re.search(r'_(\d+)_', filename)
    size = int(size_match.group(1)) if size_match else 0
    
    # Return tuple for sorting (first by model, then by size)
    return (model_name, size)

# Directory where metrics are stored
DIR = '/home/khoa/Khoa/outsource/na_thesis/examples/hello-world/ml-to-fl/pt/src/metrics'
METRIC_NAMES = ['train_metrics', 'valid_metrics']
METRIC_PERSIONS =  ['accuracy', 'precision', 'recall', 'f1_score', 'auc_roc', 'auc_pr', 'loss']

models = [FraudNet(), EnhancedFraudNet(), AttentionTransformerFraudNet()]
for model in models:
    for metric_name in METRIC_NAMES:
        for metric_percision in METRIC_PERSIONS:
            model_name = model.__class__.__name__
            
            metrics = os.listdir(DIR)
            metrics.sort()
            
            

            # Set up the plot
            plt.figure(figsize=(12, 6))
            plt.title(f'{metric_name}_{metric_percision} in {model_name}', fontsize=14)
            plt.xlabel('Step', fontsize=12)
            plt.ylabel('Metric Value', fontsize=12)

            # Colors for different models with better contrast
            colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
            markers = ['o', 's', '^', 'D', 'x', '*']

            # Load and plot each metric file
            attention_models = []
            for i, metric in enumerate(sorted(metrics, key=sort_by_model_and_size)):
                if metric.startswith(model_name):
                    attention_models.append(metric)
                    with open(os.path.join(DIR, metric), 'rb') as f:
                        data = pickle.load(f)
                            
                    # Extract x and y values for plotting
                    x_values = []
                    y_values = []
                    
                    for j, point in enumerate(data[metric_name]):
                        x_values.append(j)
                        y_values.append(point[metric_percision])
                    
                    # Plot each point individually
                    for j in range(len(x_values)):
                        plt.plot(x_values[j], y_values[j], 
                                marker=markers[i % len(markers)], 
                                color=colors[i % len(colors)],
                                markersize=5)
                    
                    # Connect points with a line
                    plt.plot(x_values, y_values, 
                            color=colors[i % len(colors)], 
                            linewidth=1.5, 
                            alpha=0.7,
                            label=f'{metric}')

            # Add legend with better placement
            plt.legend(loc='best', fontsize=10)

            # Add grid for better readability but make it subtle
            plt.grid(True, linestyle='--', alpha=0.3)

            # Improve appearance
            plt.tight_layout()

            # Save the figure
            plt.savefig(f'{model_name}_{metric_name}_{metric_percision}_pos_weight_points.png', dpi=300, bbox_inches='tight')

            # Show the plot
            plt.show()

# Same Batch Size Diff Model

In [43]:
import os
import pickle
import matplotlib.pyplot as plt
import numpy as np
import re
from matplotlib.lines import Line2D

# Directory where metrics are stored
DIR = '/home/khoa/Khoa/outsource/na_thesis/examples/hello-world/ml-to-fl/pt/src/metrics'
METRIC_NAMES = ['train_metrics', 'valid_metrics']
METRIC_PERSIONS = ['accuracy', 'precision', 'recall', 'f1_score', 'auc_roc', 'auc_pr', 'loss']

# Define batch sizes and models to compare
BATCH_SIZE = "256"  # Set this to the batch size you want to compare
MODEL_NAMES = ["FraudNet", "EnhancedFraudNet", "AttentionTransformerFraudNet"]

# Function to check if a file matches our criteria
def matches_criteria(filename, model_name, batch_size, use_pos_weight):
    # Check if file starts with the model name
    if not filename.startswith(model_name):
        return False
    
    # Check if file contains the specified batch size
    batch_match = re.search(r'_(\d+)_', filename)
    if not (batch_match and batch_match.group(1) == batch_size):
        return False
    
    # Check if file has "pos_weight" according to the preference
    has_pos_weight = "pos_weight" in filename
    if has_pos_weight != use_pos_weight:
        return False
    
    return True

# Create plots for each metric
for metric_name in METRIC_NAMES:
    for metric_precision in METRIC_PERSIONS:
        # Set up the plot
        plt.figure(figsize=(14, 8))
        plt.title(f'Comparison of Models: {metric_name}_{metric_precision} (Batch Size {BATCH_SIZE})', 
                 fontsize=14)
        plt.xlabel('Step', fontsize=12)
        plt.ylabel(f'{metric_precision.replace("_", " ").title()}', fontsize=12)

        # Colors for different models with better contrast
        colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
        
        # Different line styles for pos_weight vs regular
        line_styles = ['-', '--']  # Solid for regular, dashed for pos_weight
        markers = ['o', 's', '^', 'D', 'x', '*']

        # Get all metrics files
        metrics_files = os.listdir(DIR)
        
        # To store data for custom legend
        legend_elements = []
        
        # For each model, find and plot both the pos_weight and regular files
        for i, model_name in enumerate(MODEL_NAMES):
            color = colors[i % len(colors)]
            marker = markers[i % len(markers)]
            
            for weight_type in [False, True]:  # False = regular, True = pos_weight
                model_found = False
                for metric_file in metrics_files:
                    if matches_criteria(metric_file, model_name, BATCH_SIZE, weight_type):
                        with open(os.path.join(DIR, metric_file), 'rb') as f:
                            try:
                                data = pickle.load(f)                                
                                # Extract x and y values for plotting
                                x_values = []
                                y_values = []
                                
                                for j, point in enumerate(data[metric_name]):
                                    # Check if the metric precision exists in the point data
                                    if isinstance(point, dict) and metric_precision in point:
                                        x_values.append(j)
                                        y_values.append(point[metric_precision])
                                
                                if len(x_values) > 0:
                                    # Choose line style by weight type
                                    line_style = line_styles[1 if weight_type else 0]
                                    
                                    # Create appropriate label
                                    label = f"{model_name} {'(pos_weight)' if weight_type else '(regular)'}"
                                    
                                    # Plot the line
                                    plt.plot(x_values, y_values,
                                            color=color,
                                            linestyle=line_style,
                                            linewidth=2,
                                            alpha=0.7)
                                    
                                    # Plot the points
                                    plt.scatter(x_values, y_values,
                                              marker=marker,
                                              color=color,
                                              s=30,
                                              alpha=0.8 if weight_type else 0.6)
                                    
                                    # Create a legend element for this line style
                                    legend_elements.append(
                                        Line2D([0], [0], color=color, marker=marker, linestyle=line_style,
                                              markersize=8, label=label)
                                    )
                                    
                                    model_found = True
                                else:
                                    print(f"No data points found for {metric_precision} in {metric_file}")
                                    
                            except Exception as e:
                                print(f"Error processing {metric_file}: {e}")
                        
                        if model_found:
                            # We only need one file per model/weight combo, so break after finding the first match
                            break
                
                if not model_found:
                    weight_label = "pos_weight" if weight_type else "regular"
        
        # Add a custom legend that correctly shows line styles and markers
        if legend_elements:
            plt.legend(handles=legend_elements, loc='best', fontsize=10, ncol=2)
        else:
            plt.close()
            continue
        
        # Add grid for better readability but make it subtle
        plt.grid(True, linestyle='--', alpha=0.3)
        
        # Set background color to light gray for better contrast
        plt.gca().set_facecolor('#f8f8f8')
        
        # Improve appearance
        plt.tight_layout()
        
        # Save the figure
        plt.savefig(f'comparison_{metric_name}_{metric_precision}_batch{BATCH_SIZE}_combined.png', 
                   dpi=300, bbox_inches='tight')
                
        # Close the figure to free memory
        plt.close()

print("All plots created successfully!")

All plots created successfully!
